#### database: MySQL

pip3 install: mysql-connector

**Create database 'douban-books'**
```
CREATE DATABASE IF NOT EXISTS douban-books DEFAULT CHARSET = utf8 COLLATE = utf8_general_ci;
```

**Create table 'tag-novel'**
```
USE douban-books;

CREATE TABLE IF NOT EXISTS `tag-novel`(
   `novel_title` VARCHAR(100) NOT NULL,
   `novel_author` VARCHAR(20),
   `novel_publishing` VARCHAR(100),
   `novel_press_date` DATE,
   `novel_price` VARCHAR(100),
   `novel_img_website` VARCHAR(500),
   PRIMARY KEY ( `novel_title` )
)ENGINE=InnoDB DEFAULT CHARSET=utf8;
```

In [17]:
import mysql.connector as mariadb
from mysql.connector import Error
import urllib.request
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc.methods import media, posts
from PIL import Image

def verify_image(img_file):
     #test image
     try:
        v_image = Image.open(img_file)
        v_image.verify()
        return True;
        #is valid
        #print("valid file: "+img_file)
     except OSError:
        return False;

path = '/mnt/e/image/'
wp = Client('http://localhost/wordpress-book/xmlrpc.php', 'admin', 'admin')

try:
    # open a connection of mariadb
    mariadb_connection = mariadb.connect(
        host = 'localhost',
        user = 'root',
        password = '',
        database = 'douban_books'
    )

    # get mariadb cursor
    cursor = mariadb_connection.cursor()
    
    # get rows of table
    sql = "select * from tag_novel";
    cursor.execute(sql)
    records = cursor.fetchall()
    print("Total number of rows: ", cursor.rowcount)
    for row in records:
        filename = row[5].split('/')[-1]
        if verify_image(path+filename) == False:
            continue
        #urllib.request.urlretrieve(row[5], filename=filename)
        # prepare metadata
        data = {
                'name': filename,
                'type': 'image/jpeg',  # mimetype
        }
        # read the binary file and let the XMLRPC library encode it into base64
        with open(path+filename, 'rb') as img:
                data['bits'] = xmlrpc_client.Binary(img.read())
        response = wp.call(media.UploadFile(data))
        attachment_id = response['id']
        post = WordPressPost()
        post.title = row[0]
        post.content = row[1]+", "+row[2]+", "+row[4]
        post.post_status = 'publish'  #文章状态，不写默认是草稿，private表示私密的，draft表示草稿，publish表示发布
        post.terms_names = {
            'post_tag': ['Novel', 'New'], #文章所属标签，没有则自动创建
            'category': ['Books', 'New'] #文章所属分类，没有则自动创建
         }
        post.thumbnail = attachment_id #缩略图的id
        post.id = wp.call(posts.NewPost(post))
        
        #end for 
        
    cursor.close()
    
except Error as e:
    print("Error while connecting to mariaDB", e)
    
finally:
    # close a connection of mariadb
    if(mariadb_connection.is_connected()):
        mariadb_connection.close()
        print("mariaDB connection is closed.")
#end try

Total number of rows:  472
mariaDB connection is closed.
